In [2]:
import spacy 
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import json
import random

In [ ]:
#don't mess with the codeblock box
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       data = json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
        #if any(d.get("Municipality")=='Karnes' for d in value): 
            #print("found") 
            if not any(d.get("Municipality")== obj.get("Primary County") for d in value):
                print('not found')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Primary County")
                    }
                patterns.append(pattern)
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in pc")
        
            #print(pattern)
            #print(patterns)
    return(patterns)
patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
print(patterns)

In [66]:
#TX NER TRAINING SET 
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        #print(f)
        return(json.load(f))
    
    
def save_data (file, data):
    with open (file, "w", encoding='utf-8') as f:
       json.dump(data, f, indent=4)
       #print(data)

def create_training_data(file, type):
    #print(type)
    #print(file)
    data = load_data(file)
    #print(data)
    patterns=[]
    for key, value in data.items():
        #print(key, value)
        for obj in value: #muni loop
            #print(obj.get("Municipality"))
            pattern = {
                "label":type,
                "pattern": obj.get("Municipality")
                }
            patterns.append(pattern) # end of muni loop
        for obj in value: #primary county loop 
        #if any(d.get("Municipality")=='Karnes' for d in value): 
            #print("found") 
            if not any(d.get("Municipality")== obj.get("Primary County") for d in value):
                #print('not found')
                pattern = {
                    "label":type,
                    "pattern": obj.get("Primary County")
                    }
                patterns.append(pattern)
                #if any(d.get("Primary County")=='Karnes' for d in value): 
                    #print("found in pc")
        
            #print(pattern)
            #print(patterns)
        return(patterns)

def generate_rules(patterns):
    nlp = English()
    #ruler = EntityRuler(nlp)
    ruler = nlp.add_pipe('entity_ruler', config={"validate": True})
    ruler.add_patterns(patterns) 
    nlp.to_disk("tx_trained_ner")


def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) >0:
        results = [text, {"entities": entities}]
        #print(results)
    return (results)

patterns = create_training_data('../Corridos/data/TexasMunicipalities.json', 'GPE')
generate_rules(patterns)
#print(patterns)


nlp = spacy.load("tx_trained_ner")
ie_data={}
with open ("../Corridos/corrido corpus/ElCorridodeGregorioCortez_X.txt", "r", encoding='utf-8')as f:
    text = f.read()
    #print(text)
    segments = text.split("\n\n")[0:]
    hits=[]
    #cleaning up the lyric text. making it easier to read for the program
    for segment in segments:
        #print(segment)
        segment = segment.strip()
        segment = segment.replace("\n", " ")
        results = test_model(nlp, segment)
        #print(results)
        for result in results:
            result = (result)
            hits.append(result)
    ie_data['segment'] = hits   
    print (ie_data)

save_data("../Corridos/data/TxMunicipality_training_set.json", ie_data)

{'segment': ['In the country of Karnes,  Look what has happened;  The Major Sheriff died, Leaving Román badly wounded.', {'entities': [(18, 24, 'GPE')]}, 'He struck out for Gonzales Without showing any fear, “Follow me, cowardly rangers,  I am Gregorio Cortez.”', {'entities': [(18, 26, 'GPE')]}, 'Gregorio Cortez went out,  He went towards Laredo They decided not to follow  Because they were afraid of him.', {'entities': [(43, 49, 'GPE')]}, 'Over by El Encinal,  According to what we hear,  They made him a corral,  And he killed them another sheriff.', {'entities': [(11, 18, 'GPE')]}]}
